In [ ]:
import arcpy, os, re, numpy
import pandas as pd
from numpy.lib import recfunctions

# set your own paths


# Identify the Countries that have a coastline please note, all countries are in twice as the loop seems to be only working on every second run through
Coastal_Countries = ["ABW", "AGO", "AIA", "ALB", "ARE", "ARG", "ASM", "ATA", "ATF", "ATG", "AUS", "AZE", "BEL", "BEN", "BES", "BGD", "BGR", "BHR", "BHS", "BIH", "BLM", "BLZ", "BMU", "BRA", "BRB", "BRN", "BVT", "CAN", "CCK", "CHL", "CHN", "CIV", "CMR", "COD", "COG", "COK", "COL", "COM", "CPV", "CRI", "CUB", "CUW", "CXR", "CYM", "CYP", "DEU", "DJI", "DMA", "DNK", "DOM", "DZA", "ECU", "EGY", "ERI", "ESH", "ESP", "EST", "FIN", "FJI", "FLK", "FRA", "FRO", "FSM", "GAB", "GBR", "GEO", "GGY", "GHA", "GIB", "GIN", "GLP", "GMB", "GNB", "GNQ", "GRC", "GRD", "GTM", "GUF", "GUM", "GUY", "HMD", "HND", "HRV", "HTI", "IDN", "IND", "IOT", "IRL", "IRN", "IRQ", "ISL", "ISR", "ITA", "JAM", "JEY", "JOR", "JPN", "KAZ", "KEN", "KHM", "KIR", "KNA", "KOR", "KWT", "LBN", "LBR", "LBY", "LCA", "LKA", "LTU", "LVA", "MAF", "MAR", "MCO", "MDG", "MDV", "MEX", "MHL", "MLT", "MMR", "MNE", "MNP", "MOZ", "MRT", "MSR", "MTQ", "MUS", "MYS", "MYT", "NAM", "NCL", "NFK", "NGA", "NIC", "NIU", "NLD", "NOR", "NRU", "NZL", "OMN", "PAK", "PAN", "PCN", "PER", "PHL", "PLW", "PNG", "POL", "PRI", "PRK", "PRT", "PSE", "PYF", "QAT", "REU", "ROU", "RUS", "SAU", "SDN", "SEN", "SGP", "SGS", "SHN", "SJM", "SLB", "SLE", "SLV", "SOM", "SPM", "STP", "SUR", "SVN", "SWE", "SXM", "SYC", "SYR", "TCA", "TGO", "THA", "TKL", "TKM", "TLS", "TON", "TTO", "TUN", "TUR", "TUV", "TZA", "UKR", "UMI", "URY", "USA", "VCT", "VEN", "VGB", "VIR", "VNM", "VUT", "WLF", "WSM", "YEM", "ZAF", "GRL"]
# Sort so the countries are in alphabetical order
#Coastal_Countries = sorted(Coastal_Countries)


Landlocked = ["AFG", "AND", "ARM", "AUT", "BDI", "BFA", "BLR", "BOL", "BTN", "BWA", "CAF", "CHE", "CZE", "ETH", "HKG", "HUN", "KGZ", "LAO", "LIE", "LSO", "LUX", "MAC", "MDA", "MKD", "MLI", "MNG", "MWI", "NER", "NPL", "PRY", "RWA", "SMR", "SRB", "SSD", "SVK", "SWZ", "TCD", "TJK", "UGA", "UZB", "VAT", "XPI", "XSP", "ZMB", "ZWE"]

# GADM - XAD -> GBR (Cyprus Military Base)
# GADM - XCA -> Deleted(Caspian Sea)
# GADM - XCL -> FRA (Clipperton)
# GADM - XKO -> SRB (Kosovo)
# GADM - XNC -> CYP (North Cyprus)
# GADM - XPI Disputed (Parcel Islands)
# GADM - XSP  Disputed (Spratly Islands)
# GADM - IMN -> GBR
# GADM - ALA -> FIN
# GADM - TWN -> CHN
# GADM - HKG -> CHN
# GADM - MAC -> CHN
# EEZ - SHN, TAA, ASC -> SHN
# EEZ - CPT (Overseas) -> FRA


#Set Index
#i = 0

#Run While Loop to apply the same procedure to each Country
#while  i <  len(Coastal_Countries):


#This has been left in so that the loop can be converted to a function to run on an individual country
def CountryBoundary(i):

	# set these variables and keep if running while loo
	#Country = Coastal_Countries[i]
	#Unhash if using the function
	Country = i

	#i += 1 #Continue index
	gdb      = r"C:/Users/OsgurM/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb"          #scratch file geodatabase (LOCAL)
	outdir   = r"C:/Users/OsgurM/Documents/ArcGIS/Projects/GADM_EEZ/Output2" #Output location
	
	arcpy.env.workspace = gdb
	arcpy.env.overwriteOutput = True
	arcpy.SetLogHistory(True) 
	sr_gcs = arcpy.SpatialReference('WGS 1984') 
	arcpy.env.outputCoordinateSystem = sr_gcs
	arcpy.env.parallelProcessingFactor = "100%"

	# Displays the Country being run
	print(Country + "," + " started ", str(datetime.datetime.now().time()), str(datetime.datetime.now().date()))

#arcpy.management.Delete(gdb)

	# environment
	if not arcpy.Exists(gdb): arcpy.CreateFileGDB_management(os.path.dirname(gdb), os.path.basename(gdb))

	arcpy.MakeFeatureLayer_management(r"C:\Users\osgurm\Documents\ArcGIS\Projects\GADM_EEZ\BaseData.gdb\gadm36", "gadm_total")

	arcpy.MakeFeatureLayer_management(r"C:\Users\osgurm\Documents\ArcGIS\Projects\GADM_EEZ\BaseData.gdb\eez_v10_updated", "eez_total")

	arcpy.env.extent = None
	arcpy.env.workspace = gdb
	
	# Select the EEZ that is primarily associated with the Country
	arcpy.conversion.FeatureClassToFeatureClass("eez_total", gdb, "eez_imm", "ISO_Ter1 LIKE '%s'" % (Country))
	arcpy.management.RepairGeometry("eez_imm", "DELETE_NULL", "OGC")

	#Select all EEZ that aren part of this country
	arcpy.env.extent = 'eez_imm'
	arcpy.conversion.FeatureClassToFeatureClass("eez_total", gdb, "eez_Sur1", )
	arcpy.env.extent = None
	arcpy.conversion.FeatureClassToFeatureClass("eez_Sur1", gdb,"eez_Sur", "ISO_Ter1 <> '%s' AND ISO_Ter2 <> '%s' AND ISO_Ter3 <> '%s'" % (Country, Country, Country))
	arcpy.management.RepairGeometry("eez_Sur", "DELETE_NULL", "OGC")

	#Select the Gadm boundaries for the Country
	arcpy.conversion.FeatureClassToFeatureClass("gadm_total", gdb, "gadm_Sub", "GID_0 LIKE '%s'" % (Country))
	arcpy.management.RepairGeometry("gadm_Sub", "DELETE_NULL", "OGC")

	arcpy.analysis.Union("eez_imm;gadm_Sub", "eez_GADM_Union"  , "ALL", None, "NO_GAPS")
	arcpy.management.RepairGeometry("eez_GADM_Union", "DELETE_NULL", "OGC")

	arcpy.cartography.SimplifyPolygon("gadm_Sub", "gadm_Sub_SimplifyPolygon", "POINT_REMOVE", "1 Kilometers", "0 Unknown", "RESOLVE_ERRORS", "NO_KEEP", None)
	arcpy.management.RepairGeometry("gadm_Sub_SimplifyPolygon", "DELETE_NULL", "OGC")

	#arcpy.env.extent = 'eez_GADM_Union'
	arcpy.management.SelectLayerByLocation("eez_Sur", "WITHIN_A_DISTANCE", "gadm_Sub_SimplifyPolygon", "100 Meters", "NEW_SELECTION", "NOT_INVERT")
	arcpy.management.SelectLayerByLocation("eez_Sur", "WITHIN_A_DISTANCE", "eez_imm", "100 Meters", "ADD_TO_SELECTION", "NOT_INVERT")
	arcpy.conversion.FeatureClassToFeatureClass("eez_Sur", gdb, "eez_erase")
	arcpy.management.RepairGeometry("eez_erase", "DELETE_NULL", "OGC")
	arcpy.management.SelectLayerByAttribute("eez_Sur", "CLEAR_SELECTION", '', None)
	#arcpy.env.extent = None

	#arcpy.env.extent = 'gadm_Sub'
	arcpy.management.SelectLayerByLocation("gadm_total", "WITHIN_A_DISTANCE", "gadm_Sub_SimplifyPolygon", "100 Meters", "NEW_SELECTION", "NOT_INVERT")
	arcpy.conversion.FeatureClassToFeatureClass("gadm_total", gdb, "GADM_local")
	arcpy.management.SelectLayerByAttribute("gadm_total", "CLEAR_SELECTION", '', None)
	arcpy.management.RepairGeometry("GADM_local", "DELETE_NULL", "OGC")
	#arcpy.env.extent = None
	
	arcpy.conversion.FeatureClassToFeatureClass("GADM_local", gdb, "GADM_other", "GID_0 <> '%s'" % (Country))
	arcpy.management.RepairGeometry("GADM_other", "DELETE_NULL", "OGC")

	arcpy.analysis.PairwiseDissolve("GADM_other", "GADM_other_Diss", None, None, "MULTI_PART")	
	arcpy.management.RepairGeometry("GADM_other_Diss", "DELETE_NULL", "OGC")

	#Dissolve Gadm Layer to identify states
	arcpy.analysis.PairwiseDissolve("gadm_Sub",  "states", "GID_0;NAME_1;HASC_1", None, "SINGLE_PART")
	arcpy.management.RepairGeometry("states", "DELETE_NULL", "OGC")

	#Create single boundary for country assigned by gadm and buffer to identify any gaps between gadm and EEZ layers
	arcpy.analysis.PairwiseDissolve("gadm_Sub", "Country_diss", None, None, "MULTI_PART")
	arcpy.management.RepairGeometry("Country_diss", "DELETE_NULL", "OGC")

	arcpy.analysis.PairwiseDissolve("eez_GADM_Union", "eez_GADM_Union_Diss", None, None, "MULTI_PART")
	arcpy.management.RepairGeometry("eez_GADM_Union_Diss", "DELETE_NULL", "OGC")

	arcpy.env.extent = 'eez_GADM_Union'
	arcpy.management.SelectLayerByLocation("eez_total;gadm_total", "WITHIN_A_DISTANCE", "eez_GADM_Union_Diss", "1000 Meters", "NEW_SELECTION", "NOT_INVERT")
	arcpy.analysis.Union("gadm_total;eez_total", "Surrounding_Union", "ALL", None, "NO_GAPS")
	arcpy.management.RepairGeometry("Surrounding_Union", "DELETE_NULL", "OGC")
	arcpy.management.SelectLayerByAttribute("gadm_total", "CLEAR_SELECTION", '', None)
	arcpy.management.SelectLayerByAttribute("eez_total", "CLEAR_SELECTION", '', None)
	arcpy.env.extent = None


	# get slivers, which are land but not identified by gadm, intersect with thiessen so break at junctions
	arcpy.env.extent = 'eez_GADM_Union'
	arcpy.analysis.Union("Surrounding_Union ;Country_diss ;eez_imm", "Total_Plus_Surrounding", "ALL", None, "NO_GAPS")
	arcpy.management.RepairGeometry("Total_Plus_Surrounding", "DELETE_NULL", "OGC")
	arcpy.analysis.Erase("Total_Plus_Surrounding", "eez_total", "Total_Plus_Surrounding_EEZ_IMM", "0 Meters")
	arcpy.management.RepairGeometry("Total_Plus_Surrounding_EEZ_IMM", "DELETE_NULL", "OGC")
	arcpy.analysis.Erase("Total_Plus_Surrounding_EEZ_IMM", "gadm_total", "landnotgadm", "0 Meters")
	arcpy.management.RepairGeometry("landnotgadm", "DELETE_NULL", "OGC")
	arcpy.env.extent = None

	# create theissen polygons used to split slivers by densifying and turning boundary into points
	arcpy.edit.Densify("states", "DISTANCE", "1 Kilometers")
	arcpy.management.FeatureVerticesToPoints("states", "states_pts", "ALL")
	arcpy.management.RepairGeometry("states_pts", "DELETE_NULL", "OGC")

	# delete interior points
	arcpy.management.FeatureToLine("Country_diss", "Country_diss_FeatureToLine", None, "ATTRIBUTES")
	arcpy.management.RepairGeometry("Country_diss_FeatureToLine", "DELETE_NULL", "OGC")
	arcpy.analysis.PairwiseIntersect("states_pts;Country_diss_FeatureToLine", "states_pts_2", "ALL", None, "POINT")
	arcpy.management.RepairGeometry("states_pts_2", "DELETE_NULL", "OGC")

	# generate thiessen polygons of gadm (dissolved to state level) for intersecting with land slivers
	arcpy.env.extent = 'eez_GADM_Union_Diss'
	arcpy.analysis.CreateThiessenPolygons("states_pts_2", "states_thiessen", "ALL")
	arcpy.management.RepairGeometry("states_thiessen", "DELETE_NULL", "OGC")
	arcpy.analysis.PairwiseDissolve("states_thiessen", "theissen_mol", "GID_0;NAME_1;HASC_1", None, "SINGLE_PART")
	arcpy.management.RepairGeometry("theissen_mol", "DELETE_NULL", "OGC")
	arcpy.env.extent = None

	#Assign slivers the county information for which they fall in. Delete slivers identified that are not for the correct country and delete them
	arcpy.management.MultipartToSinglepart("landnotgadm", "slivers_all")
	arcpy.management.RepairGeometry("slivers_all", "DELETE_NULL", "OGC")
	arcpy.analysis.PairwiseIntersect("slivers_all #;eez_GADM_Union_Diss #", "slivers", "ALL", None, "INPUT")

	arcpy.analysis.PairwiseIntersect("slivers #;theissen_mol #", "slivers_int", "ALL", None, "INPUT")
	arcpy.management.RepairGeometry("slivers_int", "DELETE_NULL", "OGC")
	arcpy.analysis.PairwiseDissolve("slivers_int", "slivers_PD", "GID_01;NAME_12;HASC_12", None, "SINGLE_PART")
	arcpy.management.RepairGeometry("slivers_PD", "DELETE_NULL", "OGC")

	arcpy.management.AlterField("slivers_PD", "GID_01", "GID_0", '', "TEXT", 5000, "NULLABLE", "DO_NOT_CLEAR")
	arcpy.management.AlterField("slivers_PD", "HASC_12", "HASC_1", '', "TEXT", 5000, "NULLABLE", "DO_NOT_CLEAR")
	arcpy.management.AlterField("slivers_PD", "NAME_12", "NAME_1", '', "TEXT", 5000, "NULLABLE", "DO_NOT_CLEAR")	
		
	#Join the slivers with the states and add field which indicates that it is assigned to land
	arcpy.management.Merge("slivers_PD;states", "state_slivers")
	arcpy.management.RepairGeometry("state_slivers", "DELETE_NULL", "OGC")
	arcpy.analysis.PairwiseDissolve("state_slivers", "states_mol", "GID_0;HASC_1;NAME_1", None, "MULTI_PART")
	arcpy.management.RepairGeometry("states_mol", "DELETE_NULL", "OGC")
	arcpy.management.AddField("states_mol", "type", "TEXT", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
	arcpy.management.CalculateField("states_mol", "type", '"Land"', "PYTHON3", '')

	# get regions out to eez as full regions offshore and add field that identifies them as EEZ. (Some fields need renaming due to geoprocessing that occured earlier)
	arcpy.analysis.PairwiseIntersect("eez_imm #;theissen_mol #", "eez_imm_Local", "ALL", None, "INPUT")
	arcpy.management.RepairGeometry("eez_imm_Local", "DELETE_NULL", "OGC")
	arcpy.analysis.PairwiseDissolve("eez_imm_Local", "eez_imm_Local_PD", "GID_0;NAME_1;ISO_Ter1;ISO_Ter2;ISO_Ter3;HASC_1", None, "MULTI_PART")
	arcpy.management.RepairGeometry("eez_imm_Local_PD", "DELETE_NULL", "OGC")
	arcpy.management.AddField("eez_imm_Local_PD", "type", "TEXT", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
	arcpy.management.CalculateField("eez_imm_Local_PD", "type", '"EEZ"', "PYTHON3", '')

	#Ensure no extra area is included by deleting anything that overlaps with gadm or EEZ that isnt assigned to the country
	arcpy.env.extent = 'eez_GADM_Union'
	arcpy.analysis.PairwiseIntersect("states_mol;eez_total", "EEZ_Rem", "ONLY_FID", None, "INPUT")
	arcpy.management.RepairGeometry("EEZ_Rem", "DELETE_NULL", "OGC")
	if arcpy.management.GetCount("EEZ_Rem")[0] == "0":
		arcpy.conversion.FeatureClassToFeatureClass("states_mol", gdb, "states_mol_Erase")
		arcpy.management.RepairGeometry("states_mol_Erase", "DELETE_NULL", "OGC")

	else:
		arcpy.analysis.Erase("states_mol", "eez_total", "states_mol_Erase", "0 Meters")
		arcpy.management.RepairGeometry("states_mol_Erase", "DELETE_NULL", "OGC")

	arcpy.env.extent = None
	
	arcpy.Merge_management(['states_mol_Erase', 'eez_imm_Local_PD'], '%s/GADM_EEZ_%s.shp' % (outdir, Country))
	arcpy.management.RepairGeometry('%s/GADM_EEZ_%s.shp' % (outdir, Country), "DELETE_NULL", "OGC")


	#Clean out the File Geodatabase each iteration to keep memory lower
	print(Country + "," + " finished ", str(datetime.datetime.now().time()), str(datetime.datetime.now().date()))
	arcpy.management.Delete(gdb)



In [ ]:
CountryBoundary("AGO")

In [1]:
import arcpy, os, re, numpy
import pandas as pd
from numpy.lib import recfunctions

# set your own paths


# Identify the Countries that have a coastline please note, all countries are in twice as the loop seems to be only working on every second run through
Coastal_Countries = ["ABW", "AGO", "AIA", "ALB", "ARE", "ARG", "ASM", "ATA", "ATF", "ATG", "AUS", "AZE", "BEL", "BEN", "BES", "BGD", "BGR", "BHR", "BHS", "BIH", "BLM", "BLZ", "BMU", "BRA", "BRB", "BRN", "BVT", "CAN", "CCK", "CHL", "CHN", "CIV", "CMR", "COD", "COG", "COK", "COL", "COM", "CPV", "CRI", "CUB", "CUW", "CXR", "CYM", "CYP", "DEU", "DJI", "DMA", "DNK", "DOM", "DZA", "ECU", "EGY", "ERI", "ESH", "ESP", "EST", "FIN", "FJI", "FLK", "FRA", "FRO", "FSM", "GAB", "GBR", "GEO", "GGY", "GHA", "GIB", "GIN", "GLP", "GMB", "GNB", "GNQ", "GRC", "GRD", "GTM", "GUF", "GUM", "GUY", "HMD", "HND", "HRV", "HTI", "IDN", "IND", "IOT", "IRL", "IRN", "IRQ", "ISL", "ISR", "ITA", "JAM", "JEY", "JOR", "JPN", "KAZ", "KEN", "KHM", "KIR", "KNA", "KOR", "KWT", "LBN", "LBR", "LBY", "LCA", "LKA", "LTU", "LVA", "MAF", "MAR", "MCO", "MDG", "MDV", "MEX", "MHL", "MLT", "MMR", "MNE", "MNP", "MOZ", "MRT", "MSR", "MTQ", "MUS", "MYS", "MYT", "NAM", "NCL", "NFK", "NGA", "NIC", "NIU", "NLD", "NOR", "NRU", "NZL", "OMN", "PAK", "PAN", "PCN", "PER", "PHL", "PLW", "PNG", "POL", "PRI", "PRK", "PRT", "PSE", "PYF", "QAT", "REU", "ROU", "RUS", "SAU", "SDN", "SEN", "SGP", "SGS", "SHN", "SJM", "SLB", "SLE", "SLV", "SOM", "SPM", "STP", "SUR", "SVN", "SWE", "SXM", "SYC", "SYR", "TCA", "TGO", "THA", "TKL", "TKM", "TLS", "TON", "TTO", "TUN", "TUR", "TUV", "TZA", "UKR", "UMI", "URY", "USA", "VCT", "VEN", "VGB", "VIR", "VNM", "VUT", "WLF", "WSM", "YEM", "ZAF", "GRL"]
# Sort so the countries are in alphabetical order
#Coastal_Countries = sorted(Coastal_Countries)


Landlocked = ["AFG", "AND", "ARM", "AUT", "BDI", "BFA", "BLR", "BOL", "BTN", "BWA", "CAF", "CHE", "CZE", "ETH", "HKG", "HUN", "KGZ", "LAO", "LIE", "LSO", "LUX", "MAC", "MDA", "MKD", "MLI", "MNG", "MWI", "NER", "NPL", "PRY", "RWA", "SMR", "SRB", "SSD", "SVK", "SWZ", "TCD", "TJK", "UGA", "UZB", "VAT", "XPI", "XSP", "ZMB", "ZWE"]

# GADM - XAD -> GBR (Cyprus Military Base)
# GADM - XCA -> Deleted(Caspian Sea)
# GADM - XCL -> FRA (Clipperton)
# GADM - XKO -> SRB (Kosovo)
# GADM - XNC -> CYP (North Cyprus)
# GADM - XPI Disputed (Parcel Islands)
# GADM - XSP  Disputed (Spratly Islands)
# GADM - IMN -> GBR
# GADM - ALA -> FIN
# GADM - TWN -> CHN
# GADM - HKG -> CHN
# GADM - MAC -> CHN
# EEZ - SHN, TAA, ASC -> SHN
# EEZ - CPT (Overseas) -> FRA


#Set Index
#i = 0

#Run While Loop to apply the same procedure to each Country
#while  i <  len(Coastal_Countries):


In [2]:
Country = "IRL"
gdb      = r"C:/Users/OsgurM/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb"          #scratch file geodatabase (LOCAL)
outdir   = r"C:/Users/OsgurM/Documents/ArcGIS/Projects/GADM_EEZ/Output2" #Output location

arcpy.management.Delete(gdb)

if not arcpy.Exists(gdb): arcpy.CreateFileGDB_management(os.path.dirname(gdb), os.path.basename(gdb))
    
arcpy.MakeFeatureLayer_management(r"O:\00_data\001_AdministrativeUnits\GADM-001-GlobalAdministrativeAreas\gadm36_shp\gadm36.shp", "gadm_total")

arcpy.MakeFeatureLayer_management(r"O:\00_data\001_AdministrativeUnits\VLIZ-001-MaritimeBoundariesInclEEZ\v11_2019\eez_v11.shp", "eez_total")


arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True
arcpy.SetLogHistory(True) 
sr_gcs = arcpy.SpatialReference('WGS 1984') 
arcpy.env.outputCoordinateSystem = sr_gcs
arcpy.env.parallelProcessingFactor = "100%"

In [3]:
arcpy.conversion.FeatureClassToFeatureClass("eez_total", gdb, "eez_imm", "ISO_Ter1 LIKE '%s'" % (Country))
arcpy.management.RepairGeometry("eez_imm", "DELETE_NULL", "OGC")


<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\eez_imm'>

In [4]:
#Select all EEZ that aren part of this country
arcpy.env.extent = 'eez_imm'
arcpy.conversion.FeatureClassToFeatureClass("eez_total", gdb, "eez_Sur1", )
arcpy.env.extent = None
arcpy.conversion.FeatureClassToFeatureClass("eez_Sur1", gdb,"eez_Sur", "ISO_Ter1 <> '%s' AND ISO_Ter2 <> '%s' AND ISO_Ter3 <> '%s'" % (Country, Country, Country))
arcpy.management.RepairGeometry("eez_Sur", "DELETE_NULL", "OGC")


<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\eez_Sur'>

In [5]:
#Select the Gadm boundaries for the Country
arcpy.conversion.FeatureClassToFeatureClass("gadm_total", gdb, "gadm_Sub", "GID_0 LIKE '%s'" % (Country))
arcpy.management.RepairGeometry("gadm_Sub", "DELETE_NULL", "OGC")

<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\gadm_Sub'>

In [6]:
arcpy.analysis.Union("eez_imm;gadm_Sub", "eez_GADM_Union"  , "ALL", None, "NO_GAPS")
arcpy.management.RepairGeometry("eez_GADM_Union", "DELETE_NULL", "OGC")

<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\eez_GADM_Union'>

In [7]:
arcpy.cartography.SimplifyPolygon("gadm_Sub", "gadm_Sub_SimplifyPolygon", "POINT_REMOVE", "1 Kilometers", "0 Unknown", "RESOLVE_ERRORS", "NO_KEEP", None)
arcpy.management.RepairGeometry("gadm_Sub_SimplifyPolygon", "DELETE_NULL", "OGC")

<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\gadm_Sub_SimplifyPolygon'>

In [8]:
arcpy.management.SelectLayerByLocation("eez_Sur", "WITHIN_A_DISTANCE", "gadm_Sub_SimplifyPolygon", "100 Meters", "NEW_SELECTION", "NOT_INVERT")
arcpy.management.SelectLayerByLocation("eez_Sur", "WITHIN_A_DISTANCE", "eez_imm", "100 Meters", "ADD_TO_SELECTION", "NOT_INVERT")
arcpy.conversion.FeatureClassToFeatureClass("eez_Sur", gdb, "eez_erase")
arcpy.management.RepairGeometry("eez_erase", "DELETE_NULL", "OGC")
arcpy.management.SelectLayerByAttribute("eez_Sur", "CLEAR_SELECTION", '', None)

<Result 'eez_Sur_Layer8'>

In [9]:
arcpy.management.MinimumBoundingGeometry("eez_GADM_Union", "bounding", "ENVELOPE", "ALL", None, "NO_MBG_FIELDS")
arcpy.env.extent = 'bounding'
arcpy.analysis.Select("gadm_total", "gadm_local", '')
arcpy.env.extent = None

In [10]:
arcpy.analysis.Select("GADM_local", "GADM_other", "GID_0 <> '%s'" % (Country))
arcpy.management.RepairGeometry("GADM_other", "DELETE_NULL", "OGC")

<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\GADM_other'>

In [11]:
#Dissolve Gadm Layer to identify states
arcpy.analysis.PairwiseDissolve("gadm_Sub",  "states","GID_0;NAME_1;HASC_1" , None, "SINGLE_PART")
arcpy.management.RepairGeometry("states", "DELETE_NULL", "OGC")

<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\states'>

In [12]:
arcpy.analysis.PairwiseDissolve("gadm_Sub",  "Country_diss","" , None, "SINGLE_PART")

<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\Country_diss'>

In [13]:
arcpy.env.extent = 'eez_GADM_Union'
arcpy.management.SelectLayerByLocation("eez_total;gadm_total", "WITHIN_A_DISTANCE", "eez_GADM_Union", "1000 Meters", "NEW_SELECTION", "NOT_INVERT")
arcpy.analysis.Union("gadm_total;eez_total", "Surrounding_Union", "ALL", None, "NO_GAPS")

<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\Surrounding_Union'>

In [14]:
# get slivers, which are land but not identified by gadm, intersect with thiessen so break at junctions
arcpy.env.extent = 'eez_GADM_Union'
arcpy.management.RepairGeometry("Surrounding_Union", "DELETE_NULL", "OGC")
arcpy.analysis.Erase("Surrounding_Union", "eez_total", "Total_Plus_Surrounding_EEZ_IMM", "0 Meters")
arcpy.management.RepairGeometry("Total_Plus_Surrounding_EEZ_IMM", "DELETE_NULL", "OGC")
arcpy.analysis.Erase("Total_Plus_Surrounding_EEZ_IMM", "gadm_total", "landnotgadm", "0 Meters")
arcpy.management.RepairGeometry("landnotgadm", "DELETE_NULL", "OGC")
arcpy.env.extent = None

In [15]:
# create theissen polygons used to split slivers by densifying and turning boundary into points
arcpy.edit.Densify("states", "DISTANCE", "1 Kilometers")
arcpy.management.FeatureVerticesToPoints("states", "states_pts", "ALL")
arcpy.management.RepairGeometry("states_pts", "DELETE_NULL", "OGC")

<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\states_pts'>

In [16]:
# delete interior points
arcpy.management.FeatureToLine("Country_diss", "Country_diss_FeatureToLine", None, "ATTRIBUTES")
arcpy.management.RepairGeometry("Country_diss_FeatureToLine", "DELETE_NULL", "OGC")
arcpy.analysis.PairwiseIntersect("states_pts;Country_diss_FeatureToLine", "states_pts_2", "ALL", None, "POINT")
arcpy.management.RepairGeometry("states_pts_2", "DELETE_NULL", "OGC")

<Result 'C:/Users/osgurm/Documents/ArcGIS/Projects/GADM_EEZ/GADM_EEZ_Jupyter.gdb\\states_pts_2'>

In [17]:
# generate thiessen polygons of gadm (dissolved to state level) for intersecting with land slivers
arcpy.env.extent = 'bounding'
arcpy.analysis.CreateThiessenPolygons("states_pts_2", "states_thiessen", "ALL")
arcpy.management.RepairGeometry("states_thiessen", "DELETE_NULL", "OGC")
arcpy.analysis.PairwiseDissolve("states_thiessen", "theissen_mol", "GID_0;NAME_1;HASC_1", None, "SINGLE_PART")
arcpy.management.RepairGeometry("theissen_mol", "DELETE_NULL", "OGC")
arcpy.env.extent = None

In [ ]:
arcpy.management.MultipartToSinglepart("landnotgadm", "slivers_all")
arcpy.management.RepairGeometry("slivers_all", "DELETE_NULL", "OGC")
arcpy.management.Dice("slivers_all", "slivers_all_Dice", 10)
arcpy.analysis.PairwiseIntersect("slivers_all_Dice;gadm_Sub", "slivers_all_Sub", "ONLY_FID", None, "LINE")
arcpy.analysis.PairwiseIntersect("slivers_all_Dice;GADM_other", "slivers_all_other", "ONLY_FID", None, "LINE")


arcpy.management.CalculateGeometryAttributes("slivers_all_other", "Shape_Length LENGTH_GEODESIC", "METERS", '', "PROJCS['World_Mollweide',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Mollweide'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],UNIT['Meter',1.0]]", "SAME_AS_INPUT")
arcpy.management.CalculateGeometryAttributes("slivers_all_Sub", "Shape_Length LENGTH_GEODESIC", "METERS", '', "PROJCS['World_Mollweide',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Mollweide'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],UNIT['Meter',1.0]]", "SAME_AS_INPUT")
arcpy.management.AlterField("slivers_all_other", "Shape_Length", '', "other_length", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.AlterField("slivers_all_Sub", "Shape_Length", '', "sub_length", "DOUBLE", 8, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.JoinField("slivers_all_Dice", "OBJECTID", "slivers_all_Sub", "FID_slivers_all_Dice", "Shape_Length")
arcpy.management.JoinField("slivers_all_Dice", "OBJECTID", "slivers_all_other", "FID_slivers_all_Dice", "Shape_Length")


In [ ]:
codeblock= """
def updateValue(value): 
    if value == None: 
        return '0' 
    else: 
        return value"""

sub_Expr = "updateValue(!Shape_Length_1!)"
arcpy.management.CalculateField("slivers_all_Dice", "sub_len",sub_Expr, "PYTHON3",codeblock)
   
other_expr = "updateValue(!Shape_Length_12!)"    
arcpy.management.CalculateField("slivers_all_Dice", "other_len",other_expr , "PYTHON3", codeblock)
        

In [ ]:
arcpy.management.SelectLayerByAttribute("slivers_all_Dice", "NEW_SELECTION", "other_len < sub_len", None)
arcpy.management.SelectLayerByAttribute("slivers_all_Dice", "REMOVE_FROM_SELECTION", "sub_len = '0' And other_len = '0'", None)
arcpy.analysis.Select("slivers_all_Dice", "slivers_all_Dice_rem", '')
arcpy.analysis.PairwiseDissolve("slivers_all_Dice_rem", "slivers","", None, "SINGLE_PART")


In [ ]:
#Selection1 = arcpy.management.SelectLayerByLocation("slivers_all_Dice", "WITHIN_A_DISTANCE", "eez_imm", "1 Meters", "NEW_SELECTION", "NOT_INVERT")
#arcpy.CopyFeatures_management(Selection1, "Selection1")
#Selection2 =arcpy.management.SelectLayerByLocation("Selection1", "WITHIN_A_DISTANCE", "GADM_other", "0.1 Meters", "NEW_SELECTION", "INVERT")
#arcpy.CopyFeatures_management(Selection2, "Selection2")
#Selection3 =arcpy.management.SelectLayerByLocation("Selection2", "WITHIN_A_DISTANCE", "gadm_Sub", "0.1 Meters", "NEW_SELECTION", "NOT_INVERT")
#arcpy.CopyFeatures_management(Selection3, "slivers")


In [ ]:
arcpy.analysis.PairwiseIntersect("slivers #;theissen_mol #", "slivers_int", "ALL", None, "INPUT")
arcpy.analysis.PairwiseDissolve("slivers_int", "slivers_PD", "GID_0;NAME_12;HASC_12", None, "SINGLE_PART")
arcpy.management.RepairGeometry("slivers_PD", "DELETE_NULL", "OGC")

In [ ]:
arcpy.management.AlterField("slivers_PD", "GID_01", "GID_0", '', "TEXT", 5000, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.AlterField("slivers_PD", "HASC_12", "HASC_1", '', "TEXT", 5000, "NULLABLE", "DO_NOT_CLEAR")
arcpy.management.AlterField("slivers_PD", "NAME_12", "NAME_1", '', "TEXT", 5000, "NULLABLE", "DO_NOT_CLEAR")

In [ ]:
#Join the slivers with the states and add field which indicates that it is assigned to land
arcpy.management.Merge("slivers_PD;states", "state_slivers")
arcpy.management.RepairGeometry("state_slivers", "DELETE_NULL", "OGC")
arcpy.analysis.PairwiseDissolve("state_slivers", "states_mol", "GID_0;HASC_1;NAME_1", None, "MULTI_PART")
arcpy.management.RepairGeometry("states_mol", "DELETE_NULL", "OGC")
arcpy.management.AddField("states_mol", "type", "TEXT", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField("states_mol", "type", '"Land"', "PYTHON3", '')

In [ ]:
# get regions out to eez as full regions offshore and add field that identifies them as EEZ. (Some fields need renaming due to geoprocessing that occured earlier)
arcpy.analysis.PairwiseIntersect("eez_imm #;theissen_mol #", "eez_imm_Local", "ALL", None, "INPUT")
arcpy.management.RepairGeometry("eez_imm_Local", "DELETE_NULL", "OGC")
arcpy.analysis.PairwiseDissolve("eez_imm_Local", "eez_imm_Local_PD", "GID_0;NAME_1;ISO_Ter1;ISO_Ter2;ISO_Ter3;HASC_1", None, "MULTI_PART")
arcpy.management.RepairGeometry("eez_imm_Local_PD", "DELETE_NULL", "OGC")
arcpy.management.AddField("eez_imm_Local_PD", "type", "TEXT", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
arcpy.management.CalculateField("eez_imm_Local_PD", "type", '"EEZ"', "PYTHON3", '')

In [ ]:
#Ensure no extra area is included by deleting anything that overlaps with gadm or EEZ that isnt assigned to the country
arcpy.env.extent = 'eez_GADM_Union'
arcpy.analysis.PairwiseIntersect("states_mol;eez_total", "EEZ_Rem", "ONLY_FID", None, "INPUT")
arcpy.management.RepairGeometry("EEZ_Rem", "DELETE_NULL", "OGC")
if arcpy.management.GetCount("EEZ_Rem")[0] == "0":
    arcpy.conversion.FeatureClassToFeatureClass("states_mol", gdb, "states_mol_Erase")
    arcpy.management.RepairGeometry("states_mol_Erase", "DELETE_NULL", "OGC")

else:
    arcpy.analysis.Erase("states_mol", "eez_total", "states_mol_Erase", "0 Meters")
    arcpy.management.RepairGeometry("states_mol_Erase", "DELETE_NULL", "OGC")

arcpy.env.extent = None
arcpy.Merge_management(['states_mol_Erase', 'eez_imm_Local_PD'], '%s/GADM_EEZ_%s.shp' % (outdir, Country))
arcpy.management.RepairGeometry('%s/GADM_EEZ_%s.shp' % (outdir, Country), "DELETE_NULL", "OGC")

In [ ]:
arcpy.management.Delete(gdb)